## Formule comme à la sauce R

In [1]:
import pandas as pd
import statsmodels as sm

df = pd.read_csv('./data/Guerry.csv')
df.head()
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

,Lottery,Literacy,Wealth,Region
0,41,37,73,E
1,38,51,22,N
2,66,13,61,C
3,80,46,76,E
4,79,69,83,E


In [2]:
import statsmodels.formula.api as smf

##avec un ~ pour definir qui en fonction de qui
my_formula = "Lottery ~ Literacy"


### on definit la regression : ols = Ordinary Least Square pour optimiser la regression linaire
### (c'est le modele de regression dans R également)
mod = smf.ols(formula=my_formula, data=df)

### on estime (fit) le model
res_ols = mod.fit()
print (res_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     14.16
Date:                Fri, 23 Jul 2021   Prob (F-statistic):           0.000312
Time:                        17:36:33   Log-Likelihood:                -386.13
No. Observations:                  85   AIC:                             776.3
Df Residuals:                      83   BIC:                             781.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     64.2389      6.163     10.423      0.0

In [5]:
##pour definir un modele avec plusieurs variables, on utilise le '+'
my_full_formula = "Lottery ~ Literacy + Wealth + Region"

full_mod = smf.ols(formula=my_full_formula, data=df)

### on estime (fit) le model
full_res_ols = full_mod.fit()
print (full_res_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     6.636
Date:                Fri, 23 Jul 2021   Prob (F-statistic):           1.07e-05
Time:                        17:36:50   Log-Likelihood:                -375.30
No. Observations:                  85   AIC:                             764.6
Df Residuals:                      78   BIC:                             781.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      38.6517      9.456      4.087      

### ANOVA simple

In [6]:
### une fois que cette regression lineaire est calcule, on peut calculer une ANOVA dessus
import statsmodels.api as sm

table = sm.stats.anova_lm(full_res_ols, typ=2) # Type 2 ANOVA DataFrame
print(table)



                sum_sq    df          F    PR(>F)
Region     1499.998582   4.0   0.859236  0.492304
Literacy    342.375751   1.0   0.784485  0.378495
Wealth     8410.424430   1.0  19.270792  0.000035
Residual  34041.834158  78.0        NaN       NaN


Attention, ici df = 'degree of freedom', rien a voir avec le nom du dataframe!

### ANOVA avec interactions

In [7]:
#### ce modele avec le plus ne tient pas compte des interactions entre variables
### pour cela, il faut mettre des '*' a la place des '+' (idem dans R)
import statsmodels.api as sm

table = sm.stats.anova_lm(smf.ols(formula="Lottery ~ Literacy * Wealth * Region", data=df).fit(), typ=2) # Pour aller plus vite, en une seule ligne
print(table)



                              sum_sq    df          F    PR(>F)
Region                   1497.248916   4.0   0.780638  0.541849
Literacy                  195.882654   1.0   0.408518  0.524969
Literacy:Region          1275.329149   4.0   0.664933  0.618622
Wealth                   8104.500498   1.0  16.902148  0.000113
Wealth:Region            1276.034369   4.0   0.665301  0.618369
Literacy:Wealth            60.340597   1.0   0.125842  0.723931
Literacy:Wealth:Region    937.356225   4.0   0.488720  0.743963
Residual                31167.195053  65.0        NaN       NaN


Rq: on peut aussi specifier TOUTES les interactions que l'on veut tester en mettant
my_formula = "Lottery ~ Literacy + Region + Literacy:Region + Wealth +  Wealth:Region + Literacy:Wealth + Literacy:Wealth:Region"

ceci revient au meme que 'Literacy * Wealth * Region'

### ANOVA modele mixte (~ mesures répétées)

c'est la que ca se complique: dans la plupart des expé en neuroscience ou psycho, on effectue plusieurs mesures chez un meme individu. On parle alors de mesures repetees, ce qui complique le calcul de la statistique

On doit alors: chercher une mesure de significiativité des effets au sein de chaque individu entre differentes conditions par exemple ('fixed effect', ou facteur endogene), et à tester si ces differences au sein de chaque individu sont généralisables à un groupe, ou sont differentes entre des groupes d'individus ('random effect', ou effet exogene)

quand on cherche à combiner les deux, on parle alors de 

In [8]:

import statsmodels.api as sm
import statsmodels.formula.api as smf

data = pd.read_csv('./data/dietox.csv')
data.head()


,Unnamed: 0,Weight,Feed,Time,Pig,Evit,Cu,Litter
0,1,26.50000,NaN,1,4601,1,1,1
1,2,27.59999,5.200005,2,4601,1,1,1
2,3,36.50000,17.600000,3,4601,1,1,1
3,4,40.29999,28.500000,4,4601,1,1,1
4,5,49.09998,45.200001,5,4601,1,1,1


Description du dataframe: on a plusieurs cochons (index dans la colonne 'Pig') que l'on a pesé ('Weight') a differents temps (Time), et on cherche a voir si le temps a une influence sur la pesé, en tenant compte du fait que les plusieurs mesures sont faites sur un meme individu (index dans la colonne 'Pig')

In [9]:
md = smf.mixedlm('Weight ~ Time', data, groups=data['Pig']) 
## mixed lm: idem ols (avec formula) mais cette fois on rajoute une variable groups, qui indique le facteur de fixed effect
mdf = md.fit()
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Weight    
No. Observations: 861     Method:             REML      
No. Groups:       72      Scale:              11.3669   
Min. group size:  11      Log-Likelihood:     -2404.7753
Max. group size:  12      Converged:          Yes       
Mean group size:  12.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    15.724    0.788  19.952 0.000 14.179 17.268
Time          6.943    0.033 207.939 0.000  6.877  7.008
Group Var    40.394    2.149                            

